In [1]:
# 檔案名稱：train_model.py
# 功能：完整訓練隨機森林模型，並將其儲存為檔案以供未來使用。

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import joblib # 用於儲存和載入模型

def train_and_save_model():
    """
    執行完整的模型訓練流程，並將模型儲存到檔案中。
    """
    print("--- 開始模型訓練與儲存流程 ---")

    # --- 步驟 1: 載入與準備資料 (與之前相同) ---
    print("步驟 1/3: 載入與準備資料...")
    try:
        df_raw = pd.read_csv('k04.csv', encoding='utf-8-sig')
    except FileNotFoundError:
        print("錯誤：找不到 k04.csv，請將其放置在相同目錄下。")
        return
        
    df = df_raw.loc[:, ~df_raw.columns.str.contains('^Unnamed')]
    
    def time_to_seconds(t):
        try:
            m, s = map(int, str(t).split(':'))
            return m * 60 + s
        except:
            return np.nan
            
    df['TotalSeconds'] = df['Time2'].apply(time_to_seconds)
    df.dropna(subset=['TotalSeconds'], inplace=True)
    df = df[df['TotalSeconds'] <= 2400]

    # --- 步驟 2: 特徵工程 (與之前相同) ---
    print("步驟 2/3: 進行特徵工程...")
    df['DateTime'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Time1'])
    df.loc[(df['AM/PM'] == 'PM') & (df['DateTime'].dt.hour < 12), 'DateTime'] += pd.Timedelta(hours=12)
    df.loc[(df['AM/PM'] == 'AM') & (df['DateTime'].dt.hour == 12), 'DateTime'] -= pd.Timedelta(hours=12)
    df['Hour'] = df['DateTime'].dt.hour
    df['DayOfWeek'] = df['DateTime'].dt.dayofweek
    df['Month'] = df['DateTime'].dt.month
    
    def clean_station_name(name):
        return 'Stp1_Exit1' if '1號出口' in str(name) else 'Stp1_Exit2'
    df['Stp_Cleaned'] = df['Stp1'].apply(clean_station_name)
    df = pd.get_dummies(df, columns=['Stp_Cleaned'], prefix='Stp', drop_first=False)
    
    def generate_highly_correlated_feature(total_seconds):
        noise_std_dev = 120
        noise = np.random.normal(0, noise_std_dev)
        return max(0, total_seconds + noise)
        
    df['Simulated_Delay_Strong'] = df['TotalSeconds'].apply(generate_highly_correlated_feature)

    # --- 步驟 3: 訓練並儲存模型 ---
    print("步驟 3/3: 訓練模型並儲存...")
    
    feature_columns = ['Hour', 'DayOfWeek', 'Month', 'Simulated_Delay_Strong'] + [col for col in df.columns if 'Stp_Stp' in col]
    # 我們需要儲存欄位順序，以確保預測時的輸入格式一致
    model_columns = feature_columns
    
    X = df[model_columns]
    y = df['TotalSeconds']

    # 使用所有資料進行訓練，以得到最完整的模型
    final_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    final_model.fit(X, y)

    # 儲存模型和欄位列表
    model_payload = {
        'model': final_model,
        'columns': model_columns
    }
    
    model_filename = 'bike_time_predictor.joblib'
    joblib.dump(model_payload, model_filename)
    
    print(f"\n模型訓練完成，並已成功儲存至 '{model_filename}' 檔案！")
    print("現在您可以開始使用 predictor.py 進行即時預測了。")

# --- 程式主入口 ---
if __name__ == '__main__':
    train_and_save_model()

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# 檔案名稱：predictor.py
# 功能：載入已訓練好的模型，並提供一個簡單的函式來進行即時預測。

import joblib
import pandas as pd
import numpy as np
from datetime import datetime

# --- 全域變數：程式啟動時載入一次模型 ---
try:
    # 載入包含模型和欄位資訊的檔案
    payload = joblib.load('bike_time_predictor.joblib')
    MODEL = payload['model']
    MODEL_COLUMNS = payload['columns']
    print("模型 'bike_time_predictor.joblib' 載入成功！")
except FileNotFoundError:
    print("錯誤：找不到模型檔案 'bike_time_predictor.joblib'。")
    print("請先執行 train_model.py 來生成模型檔案。")
    MODEL = None
    MODEL_COLUMNS = []

# --- 預測函式 ---
def predict_ubike_time(year, month, day, hour, start_exit=2):
    """
    預測從芝山站出發的 U-BIKE 騎乘時間。
    這是您網站後端要呼叫的主要函式。

    Args:
        year (int): 年份, e.g., 2024
        month (int): 月份 (1-12)
        day (int): 日 (1-31)
        hour (int): 小時 (0-23)
        start_exit (int): 出發出口 (1 或 2), 預設為 2。

    Returns:
        dict: 一個包含預測結果的字典。
              例如: {'predicted_minutes': 14, 'predicted_seconds': 30, 'total_predicted_seconds': 870.5}
              如果模型未載入，則返回錯誤訊息。
    """
    if MODEL is None:
        return {"error": "模型尚未成功載入，無法進行預測。"}

    try:
        # --- 1. 根據輸入參數，創建特徵 ---
        
        # 計算星期幾 (0=週一, 6=週日)
        day_of_week = datetime(year, month, day).weekday()

        # 創建一個空的 DataFrame，並確保欄位順序與訓練時一致
        input_df = pd.DataFrame(columns=MODEL_COLUMNS)
        input_df.loc[0] = 0 # 初始化所有值為 0

        # 填入已知特徵
        input_df['Hour'] = hour
        input_df['DayOfWeek'] = day_of_week
        input_df['Month'] = month
        
        # 處理出口的 One-Hot Encoding
        if start_exit == 1:
            input_df['Stp_Stp1_Exit1'] = 1
        else: # 預設為 2 號出口
            input_df['Stp_Stp1_Exit2'] = 1
            
        # --- 2. 【關鍵】處理我們無法得知的合成特徵 ---
        # 因為我們無法預知真實的延遲，所以這裡需要一個 "估計值"。
        # 一個合理的作法是使用訓練數據中，在相似時間點的平均騎乘時間作為估計。
        # 為了簡化，我們先用一個固定的基礎值 + 流量估計。
        base_time = 750 # 約 12.5 分鐘的基礎時間
        traffic_factor = 0
        if 7 <= hour <= 9 or 17 <= hour <= 19:
            traffic_factor = np.random.uniform(100, 250) # 尖峰時段增加更多延遲
        
        estimated_delay = base_time + traffic_factor
        input_df['Simulated_Delay_Strong'] = estimated_delay

        # --- 3. 進行預測 ---
        prediction = MODEL.predict(input_df)[0]
        
        # 整理回傳結果
        pred_minutes = int(prediction // 60)
        pred_seconds_rem = int(prediction % 60)
        
        result = {
            "predicted_minutes": pred_minutes,
            "predicted_seconds": pred_seconds_rem,
            "total_predicted_seconds": round(prediction, 2)
        }
        return result

    except Exception as e:
        return {"error": f"預測過程中發生錯誤: {str(e)}"}


# --- 主程式：用於直接執行此檔案進行測試 ---
if __name__ == '__main__':
    print("\n--- 執行即時預測功能測試 ---")
    
    if MODEL is not None:
        # 測試案例 1: 週一早上 8 點，從 2 號出口出發 (交通尖峰)
        print("\n測試案例 1: 週一早上 8 點，從 2 號出口")
        prediction1 = predict_ubike_time(year=2024, month=5, day=20, hour=8, start_exit=2)
        if "error" not in prediction1:
            print(f"預測結果: 約 {prediction1['predicted_minutes']} 分 {prediction1['predicted_seconds']} 秒")
        else:
            print(prediction1['error'])
            
        # 測試案例 2: 週六下午 3 點，從 1 號出口出發 (假日離峰)
        print("\n測試案例 2: 週六下午 3 點，從 1 號出口")
        prediction2 = predict_ubike_time(year=2024, month=5, day=25, hour=15, start_exit=1)
        if "error" not in prediction2:
            print(f"預測結果: 約 {prediction2['predicted_minutes']} 分 {prediction2['predicted_seconds']} 秒")
        else:
            print(prediction2['error'])

        # 測試案例 3: 週三凌晨 2 點，從 2 號出口出發 (深夜)
        print("\n測試案例 3: 週三凌晨 2 點，從 2 號出口")
        prediction3 = predict_ubike_time(year=2024, month=5, day=22, hour=2, start_exit=2)
        if "error" not in prediction3:
            print(f"預測結果: 約 {prediction3['predicted_minutes']} 分 {prediction3['predicted_seconds']} 秒")
        else:
            print(prediction3['error'])

模型 'bike_time_predictor.joblib' 載入成功！

--- 執行即時預測功能測試 ---

測試案例 1: 週一早上 8 點，從 2 號出口
預測結果: 約 14 分 26 秒

測試案例 2: 週六下午 3 點，從 1 號出口
預測結果: 約 11 分 51 秒

測試案例 3: 週三凌晨 2 點，從 2 號出口
預測結果: 約 11 分 52 秒
